In [1]:
import numpy as np

In [ ]:
class Node:
         
    def __init__(self, initial, parent=None, acao=None, custo=0):

        self.initial = initial
    
    def __repr__(self):
        return "<Node {}>".format(self.initial)

    def expansao(self, problema, rainha):
        
        return [self.child_node(problema, rainha, acao) 
                for acao in problema.pos_mov()]
        

    def child_node(self, problem, rainha, acao):

        prox_estado = problem.jogada(self.initial, rainha, acao)

        prox_node = Node(prox_estado, self, acao, 0)

        return prox_node

In [ ]:
class oitorainhas:

    def __init__(self, x, y, tabuleiro=None):
        self.x = x
        self.y = y

        if tabuleiro == None:
            self.tabuleiro = np.zeros([x,y])

            for i in range(x):
                self.tabuleiro[i][0] = 1

    def pos_mov(self):

        pos = []
        for i in range(self.x):
            for j in range(self.y):
                if self.tabuleiro[i][j] != 1:
                    pos.append([i,j])
        
        return pos

    def jogada(self, rainha, coordenada):

        tab = self.tabuleiro.copy()

        tab[rainha[0],rainha[1]] = 0
        tab[coordenada[0],coordenada[1]] = 1

        return tab